In [1]:
# ultimate goal: 
# get the trial solution psi_t(x) = A_D(x) + L_D(x)L_M(x)g_M(x) + L_D(x)N(x)

In [2]:
class ControlPoint:
    
    def __init__(self, x, psi=None, nx=None, dpsi=None, dim=2):
        """
        psi(x) is the function we are solving for
        
        :param x: coordinate of this control point
        :param psi: (only for Dirichlet B.C.) psi(x)
        :param nx: (only for Neumann B.C.) normal vector of this control point
        :param dpsi: (only for Neumann B.C.) nx dot grad(psi(x)) = dpsi
        :param dim: dimension of this control point
        """

In [3]:
class Interpolator:
    
    """Methods here can be write on top of some general thin plate spline implementation?"""
    
    def fit_length_factor(control_points):
        """
        list of ControlPoints -> length factor function:
        if control points are for Dirichlet B.C. then L_D(x)
        if control points are for Neumann B.C. then L_M(x)
        """
        # for 2D boundaries:
        # sort the control_points by their theta (in polar cordinate)
        # assign each point the mapped point (equally spaced on a circle)
        # solve the coefficients F for thin plate spline
        # return L_D(x)/L_M(x) parameterized by F
        
        # for 3D boundaries:
        # how to sort the control_points?
        # how to assign mapped point? (what does equally spaced mean on a sphere?)
    
    def fit_dirichlet_bondary(control_points):
        """
        the A_D(x) in psi_t(x) = A_D(x) + L_D(x)L_M(x)g_M(x) + L_D(x)N(x)
        """
        # again a thin plate spline of the Dirichlet coundary control points? (re-check the paper)
        
    def fit_normal_vector(control_points):
        """
        the g_M(x) in psi_t(x) = A_D(x) + L_D(x)L_M(x)g_M(x) + L_D(x)N(x)
        involves the function n(x) that map x to the normal vector at that point,
        this method fit that n(x)
        """
        # again a thin plate spline of the Neumann coundary control points? (re-check the paper)

In [4]:
class GeneralBoundaryCondition:
    
    def __init__(self, control_points, ith):
        """
        control_points should include both Neumann B.C. and Dirichlet B.C.
        maybe should create different class for them?
        """
        # fit L_D(x), L_M(x), A_D(x), n(x)
        # use the above functions to construct g_M(x)
        # construct the final trial solution
    
    def _fit_g_M(L_D, L_M, A_D, nx):
        """
        fit g_M(x), called in the initializer
        maybe this method should be in interpolator?
        """
    
    def enforce(self, net, xyz_etc):
    
    def set_impose_on(self, ith):
        """
        conform to the new interface of condition class
        maybe I should have a base class so that we are not writing this method over and over again...
        """